<a href="https://colab.research.google.com/github/kae1dy/NLPCodeReview/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install transformers

from datasets import Dataset
import pandas as pd
from transformers import AutoModel, AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.4 MB/s eta 0:00:00


In [2]:
# preprocessing dataset

url = "https://raw.githubusercontent.com/CommentFinder/CommentFinder/master/dataset"

data_files = {
    "train": url + "/train.tsv",
    "test":  url + "/test.tsv",
}
train = pd.read_csv(data_files["train"], header=None, sep='\t', on_bad_lines='skip', skipinitialspace=True, names=['source', 'target'])
test = pd.read_csv(data_files["test"], header=None, sep='\t', on_bad_lines='skip', skipinitialspace=True, names=['source', 'target'])

train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test[:5000])

train, test

(Dataset({
     features: ['source', 'target'],
     num_rows: 134225
 }),
 Dataset({
     features: ['source', 'target'],
     num_rows: 5000
 }))

In [3]:
checkpoint = "Salesforce/codet5p-110m-embedding"
device = "cuda"

import gc
gc.collect()

tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

# code embedding
def get_embedding(source):
    input = tokenizer(source, padding=True, truncation=True, return_tensors="pt") # truncation=True
    input = {k: v.to(device) for k, v in input.items()}

    model_output = model(**input)
    return model_output

train = train.map(
    lambda x: {"embedding": get_embedding(x["source"]).detach().cpu().numpy()[0]}, remove_columns=["source"] # batched=True, batch_size=26, remove_columns=["source", "target"]
)
test = test.map(
    lambda x: {"embedding": get_embedding(x["source"]).detach().cpu().numpy()[0]}, remove_columns=["source"] # batched=True, batch_size=26, remove_columns=["source", "target"]
)

# with batched 30.83 examples/s
# without 55.29 examples/s
train, test

A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-110m-embedding:
- configuration_codet5p_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/Salesforce/codet5p-110m-embedding:
- modeling_codet5p_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Map:   0%|          | 0/134225 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

(Dataset({
     features: ['target', 'embedding'],
     num_rows: 134225
 }),
 Dataset({
     features: ['target', 'embedding'],
     num_rows: 5000
 }))

In [4]:
import statistics
import time
import numpy as np
from nltk.translate import bleu_score
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from functools import wraps

# train['embedding'], test['embedding']
print(f'\nVector Length: {len(test["embedding"][0])}.')


def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        total_time = time.perf_counter() - start_time
        print(f'\nTime cost ({func.__name__}): {total_time:.4f} seconds.')
        return result
    return timeit_wrapper


def predict_top_k(test, source, topk=10) -> list:
    prediction = []
    similarity = cosine_similarity(test['embedding'], source['embedding'])

    for index, similar in enumerate(tqdm(similarity)):

        index_nn = np.argpartition(similar, -topk)[-topk:]
        current_prediction = [source['target'][i] for i in index_nn]

        prediction.append(current_prediction)

    #   write the recommendation comments to the file named as "our_predictions_k.txt"
    with open('predictions_' + str(topk) + '.txt', 'w') as f:
        for data in prediction:
            for element in data:
                f.write(element + '\n')
    return prediction


def batch_data(data, batch_size=1000):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]


chencherry = bleu_score.SmoothingFunction()
BLEUscore = []
count_perfect = 0
k = 10

for test_batch in batch_data(test):
    # Compute the cosine distance and its computational time
    prediction = predict_top_k(test_batch, train, k)

    # Evaluate perfect prediction & BLEU score of our approach
    for i, target in enumerate(tqdm(test_batch['target'])):
        best_BLEU = 0
        for pred in prediction[i]:
            current_BLEU = bleu_score.sentence_bleu([target], pred, smoothing_function=chencherry.method1)
            best_BLEU = max(best_BLEU, current_BLEU)

            if " ".join(pred.split()) == " ".join(target.split()):
                count_perfect += 1
                break
        BLEUscore.append(best_BLEU)


print(f'\nPP    : %d/%d (%s%.2f)' % (count_perfect, len(test['target']), '%', (count_perfect * 100) / len(test['target'])))
print(f'BLEU mean              : ', statistics.mean(BLEUscore))


Vector Length: 256.


100%|██████████| 1000/1000 [00:04<00:00, 246.67it/s]


PP    : 204/5000 (%4.08)
BLEU mean              :  0.24526899805535846
